In [1]:
### Import Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pprint
from tabulate import tabulate
import warnings

%matplotlib inline
pd.set_option("display.max_columns",40)
warnings.filterwarnings("ignore")

from IPython.display import HTML


import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier


from sklearn.metrics import confusion_matrix,accuracy_score,auc,roc_curve,f1_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler,binarize

from mlxtend.classifier import StackingClassifier
from xgboost import XGBClassifier
from vecstack import stacking
from pandas_ml import ConfusionMatrix

Using TensorFlow backend.


In [2]:
def get_roc_curve(model,x_test,y_test):
    prob=model.predict_proba(x_test)
    preds=prob[:,1]
    fpr, tpr, threshold = roc_curve(y_test, preds)
    roc_auc =auc(fpr, tpr)
    plt.figure(figsize=(10,8))
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(threshold, index = i)})
    threshold=roc.ix[(roc.tf-0).abs().argsort()[:1]]['thresholds'].values[0]
    print("Optimal Threshold is",threshold)
    y_pred_class=binarize(preds.reshape(1,-1),threshold)[0]
    accuracy=accuracy_score(y_test,y_pred_class)
    print("Accuracy of Logistic Model on training data is:",accuracy)
    cm=ConfusionMatrix(y_test,y_pred_class)
    display(cm)
    return cm,accuracy,threshold,roc_auc

In [3]:
# # ## Read the csv file which contains the data aggregated on user level 
# # z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_0.csv',index_col='Source_User')

# # z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
# #     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
# #     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# # ]


# # columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
# #     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
# #     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# # ]

# # print("The feature vector for infected users and sample of uninfected users")
# # print('Infected/Unifected Users')
# # display(z.Target.value_counts())
# # ## Create a feature set
# # X=z.iloc[:,z.columns!='Target']
# # display(X.head(2))
# # X=X.values

# # ### Build our target variable
# # print("1 -> Infected\n0 -> Uninfected")
# # Y=z.Target
# # Y=Y.apply(lambda x: 1 if x=='infected' else 0)
# # display(Y.head(10))
# # Y=Y.values

# # ### Scale feature set  using Standard Scaler
# # from sklearn.preprocessing import StandardScaler
# # scaler=StandardScaler()
# # scaled_X=scaler.fit_transform(X)
# # print("The train/test split ratio is 80:20")
# # from sklearn.model_selection import train_test_split
# # X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)


# # clf1 = BaggingClassifier(n_estimators=300, max_samples=0.6,max_features=0.4,
# #                       bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)
# # clf2 =  GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.3,
# #                                min_samples_leaf=20,subsample=0.5,random_state = 0, n_estimators = 150)
# # clf3 = AdaBoostClassifier(n_estimators=50,learning_rate=1.5,algorithm='SAMME',random_state=0)
# # lr = LogisticRegression()
# # sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
# #                           use_probas=True,
# #                           average_probas=True,
# #                           meta_classifier=lr)

# # print('3-fold cross validation:\n')

# # for clf, label in zip([clf1, clf2, clf3, sclf], 
# #                       ['KNN', 
# #                        'Random Forest', 
# #                        'Naive Bayes',
# #                        'StackingClassifier']):

# #     scores = cross_val_score(clf, X_train,Y_train,cv=3, scoring='accuracy')
# #     print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
# #           % (scores.mean(), scores.std(), label))

# # sclf.fit(X_train,Y_train)
# # acc_test=accuracy_score(Y_test,sclf.predict(X_test))
# # cm_test=ConfusionMatrix(Y_test,sclf.predict(X_test))
# # print(acc_test)
# # print(cm_test)

# # acc_train=sclf.score(X_train,Y_train)
# # cm_train=ConfusionMatrix(Y_train,sclf.predict(X_train))
# # print(acc_train)
# # print(cm_train)

# # cm,accuracy,threshold,roc_auc=get_roc_curve(sclf,X_test,Y_test)

# # t=pd.DataFrame([accuracy, cm.TP,      cm.TN,      cm.FP,      cm.FN,      cm.FPR,      cm.FNR, threshold,roc_auc]).transpose()
# # t.head()



# ## Read the csv file which contains the data aggregated on user level 
# z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_10.csv',index_col='Source_User')

# z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
#     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
#     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# ]


# columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
#     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
#     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# ]

# print("The feature vector for infected users and sample of uninfected users")
# print('Infected/Unifected Users')
# display(z.Target.value_counts())
# ## Create a feature set
# X=z.iloc[:,z.columns!='Target']
# display(X.head(2))
# X=X.values

# ### Build our target variable
# print("1 -> Infected\n0 -> Uninfected")
# Y=z.Target
# Y=Y.apply(lambda x: 1 if x=='infected' else 0)
# display(Y.head(10))
# Y=Y.values

# ### Scale feature set  using Standard Scaler
# from sklearn.preprocessing import StandardScaler
# scaler=StandardScaler()
# scaled_X=scaler.fit_transform(X)
# print("The train/test split ratio is 80:20")
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)


# clf1 = BaggingClassifier(n_estimators=300, max_samples=0.6,max_features=0.4,
#                       bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)
# clf2 =  GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.3,
#                                min_samples_leaf=20,subsample=0.5,random_state = 0, n_estimators = 150)
# clf3 = AdaBoostClassifier(n_estimators=50,learning_rate=1.5,algorithm='SAMME',random_state=0)
# lr = LogisticRegression()
# sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
#                           use_probas=True,
#                           average_probas=True,
#                           meta_classifier=lr)

# print('3-fold cross validation:\n')

# for clf, label in zip([clf1, clf2, clf3, sclf], 
#                       ['KNN', 
#                        'Random Forest', 
#                        'Naive Bayes',
#                        'StackingClassifier']):

#     scores = cross_val_score(clf, X_train,Y_train,cv=3, scoring='accuracy')
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
#           % (scores.mean(), scores.std(), label))

# sclf.fit(X_train,Y_train)
# acc_test=accuracy_score(Y_test,sclf.predict(X_test))
# cm_test=ConfusionMatrix(Y_test,sclf.predict(X_test))
# print(acc_test)
# print(cm_test)

# acc_train=sclf.score(X_train,Y_train)
# cm_train=ConfusionMatrix(Y_train,sclf.predict(X_train))
# print(acc_train)
# print(cm_train)

# cm,accuracy,threshold,roc_auc=get_roc_curve(sclf,X_test,Y_test)

# t=pd.DataFrame([accuracy, cm.TP,      cm.TN,      cm.FP,      cm.FN,      cm.FPR,      cm.FNR, threshold,roc_auc]).transpose()
# t.head()







# ## Read the csv file which contains the data aggregated on user level 
# z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_20.csv',index_col='Source_User')

# z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
#     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
#     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# ]


# columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
#     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
#     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# ]

# print("The feature vector for infected users and sample of uninfected users")
# print('Infected/Unifected Users')
# display(z.Target.value_counts())
# ## Create a feature set
# X=z.iloc[:,z.columns!='Target']
# display(X.head(2))
# X=X.values

# ### Build our target variable
# print("1 -> Infected\n0 -> Uninfected")
# Y=z.Target
# Y=Y.apply(lambda x: 1 if x=='infected' else 0)
# display(Y.head(10))
# Y=Y.values

# ### Scale feature set  using Standard Scaler
# from sklearn.preprocessing import StandardScaler
# scaler=StandardScaler()
# scaled_X=scaler.fit_transform(X)
# print("The train/test split ratio is 80:20")
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)


# clf1 = BaggingClassifier(n_estimators=300, max_samples=0.6,max_features=0.4,
#                       bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)
# clf2 =  GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.3,
#                                min_samples_leaf=20,subsample=0.5,random_state = 0, n_estimators = 150)
# clf3 = AdaBoostClassifier(n_estimators=50,learning_rate=1.5,algorithm='SAMME',random_state=0)
# lr = LogisticRegression()
# sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
#                           use_probas=True,
#                           average_probas=True,
#                           meta_classifier=lr)

# print('3-fold cross validation:\n')

# for clf, label in zip([clf1, clf2, clf3, sclf],['KNN', 'Random Forest', 'Naive Bayes','StackingClassifier']):

#     scores = cross_val_score(clf, X_train,Y_train,cv=3, scoring='accuracy')
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
#           % (scores.mean(), scores.std(), label))

# sclf.fit(X_train,Y_train)
# acc_test=accuracy_score(Y_test,sclf.predict(X_test))
# cm_test=ConfusionMatrix(Y_test,sclf.predict(X_test))
# print(acc_test)
# print(cm_test)

# acc_train=sclf.score(X_train,Y_train)
# cm_train=ConfusionMatrix(Y_train,sclf.predict(X_train))
# print(acc_train)
# print(cm_train)

# cm,accuracy,threshold,roc_auc=get_roc_curve(sclf,X_test,Y_test)

# t=pd.DataFrame([accuracy, cm.TP,      cm.TN,      cm.FP,      cm.FN,      cm.FPR,      cm.FNR, threshold,roc_auc]).transpose()
# t.head()

In [4]:
cric=['eee73a52c55dbef6c5b4ec7ec0cd3d200a90f528b7344b7337a546f7734774db',
       '1510909c55f8994397a5aced5728fca609aff1b6bab31080e919795aeb2db97d',
       'cff22bd1423ddbbbf00818b3d866cb4a5f4a8603ee607a824ad96224072f4076',
       '1ed642700929c99329b1ec5e6ddaa5b55db8ec04f776a690e4e22b86e89136db',
       'c96be1bc82e9c8d37ba23c7077d47f003a88fe0b5e0ef79ffa830d9552eeed1e',
       '61066be7a03d4212024695c0d434a3f6cf94f81d89911b6f5cd9c96819b0b064',
       'a2d8c624dbcb266d862fbbfb3920c06f051b4c78c8d8f033ccea74b92c433112',
       'f8bb4b13cb09963f5c4e51af61a1946eb2dbda71188d1e203d14163868ac5752',
       'cca0d130b4966f6bcc4e7093295e2637b48f421c120f4d6bfece4938e1c61ae7',
       'cf4dffb6fb12b9a95d1cc43717950e389db7e3c7c9e6c4a3d56d8e0c7a98891b',
       'bc87724fed1de64476927e18776a43eea899dea2e2a39ba6bf69dfe1eaf8b145',
       'db663ee7c30f6e366215d111f745205c73789819f5781028b573a197b0388e23',
       '5b2109fa7d9154dc8b15a1ecee9f520330519e9e4d8ea51a61e01cbc44600a86',
       '5672b291263e608577995d50cd51efd4662af2b9a3d327db7d6c264112bd7d57',
       '6d0095effd2435ab7c4557e92698c57345ac103d4668cf902221faec2052d1ea',
       '4121c37e4f4081f7e533940466b0962159f00840b14dca4479eb17e0d5807b15',
       '996116b7d5b47888082dd8caf8911ef352ebe6311210d813fbc18710e12a0091']

In [5]:
crit=['eee73a52c55dbef6c5b4ec7ec0cd3d200a90f528b7344b7337a546f7734774db',
       '1510909c55f8994397a5aced5728fca609aff1b6bab31080e919795aeb2db97d',
       'cff22bd1423ddbbbf00818b3d866cb4a5f4a8603ee607a824ad96224072f4076',
       '1ed642700929c99329b1ec5e6ddaa5b55db8ec04f776a690e4e22b86e89136db',
       'c96be1bc82e9c8d37ba23c7077d47f003a88fe0b5e0ef79ffa830d9552eeed1e',
       '61066be7a03d4212024695c0d434a3f6cf94f81d89911b6f5cd9c96819b0b064',
       'a2d8c624dbcb266d862fbbfb3920c06f051b4c78c8d8f033ccea74b92c433112',
       'f8bb4b13cb09963f5c4e51af61a1946eb2dbda71188d1e203d14163868ac5752',
       'cca0d130b4966f6bcc4e7093295e2637b48f421c120f4d6bfece4938e1c61ae7',
       'cf4dffb6fb12b9a95d1cc43717950e389db7e3c7c9e6c4a3d56d8e0c7a98891b',
       'bc87724fed1de64476927e18776a43eea899dea2e2a39ba6bf69dfe1eaf8b145',
       'db663ee7c30f6e366215d111f745205c73789819f5781028b573a197b0388e23',
       '5b2109fa7d9154dc8b15a1ecee9f520330519e9e4d8ea51a61e01cbc44600a86',
       '5672b291263e608577995d50cd51efd4662af2b9a3d327db7d6c264112bd7d57',
       '6d0095effd2435ab7c4557e92698c57345ac103d4668cf902221faec2052d1ea',
       '4121c37e4f4081f7e533940466b0962159f00840b14dca4479eb17e0d5807b15',
       '996116b7d5b47888082dd8caf8911ef352ebe6311210d813fbc18710e12a0091']

med=['07a805db92f77828144af02c110b4107381c8608ff69cbe8322b1619550dca26',
       'eee73a52c55dbef6c5b4ec7ec0cd3d200a90f528b7344b7337a546f7734774db',
       'cef0d1cb38af8ad44a4c03c5718984ae948de73fefefc7f16e8e87a27bd719ff',
       '1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900',
       '335e6cc2247d9cbd022e95cff5b6d3bd1da43b40cc993c2b840800394bb569b4',
       '98a76f9142fc866cdad367b1ea486dbb062fa0782e92e7eddf470385af2b5484',
       'b1d81f7de972e44a6845b9556ed3485a9b9a4a27c499d707ca91d3f5aa2334cc',
       '5c95fd730ce2612f44cb7a9c5802d9c5c9e90d7efdeb81026d8a89ab5a3c0a8a',
       '9861528d3962a11deb48775c63026bc8cbdf39aaf3f90aa0b87fef5445d5926c',
       'ce91776ab66403168967f46d8170b6949035822ba671e2ae1a4e5b012f71fc73',
       '42ee4c8133818830b3d084683b96a8209ca6b0d6112f25d0002681474301404c',
       'b0de67ec800462fd2105c2c142ae87db886e09c6eb0a1d33448afbf25fb57dfa',
       'f11e0185ff5a19066b53c1f397619e29796fd2f4623ac97a54764213cd997ccc',
       'd4f33d03ecd342a343c01d55c5b1191e37b1bc77452e20516b52cc6de0611f33',
       '26490d34b73ba281b2a586fdbfc9d13b48deaa204944733b859bf412f1fa9b0e',
       '2b9d80a899a0e34177f2b0ad451f18509d8b73540ce2b51fc2dc652495c99417',
       '95fef8a0d3377bb3fb41d589e5091eece852026be1b1224383265748863b6671',
       '0caa319489c018ddc7862bf2dafc054889ec8066de3f5b166d1a26e9f12920a4',
       '9bf55dee2eb78fca489c15f4c05644d7e259c413f4f97ab1d622e9999703ff7a',
       '94e77fdd5917ebf29fc743bf64798a5dd7c60068fcd029559e4e03afae2cda39',
       '8b7d9bb36cbb3b6d3defe024d4b980f5bedb71d8875456642e2b173dede2a3c7',
       'f71994f88f63bd783f50648de55e879dc8580e20c771703b852c0e3036667f46',
       'f52846069af3ef50f230f86ac40cdfe41e461a5e749951f112cd0f7c8e605a9e',
       '4677f4c9612858664db2eac9f788886ba53d3b48d79320c1fb15aaf04e00d970',
       '201b65b57be8702dfa4593b39f0d5a9d79662627dc885199c1373b188d4a4a47',
       'bb445f8ffb0724b780364707b75ade0df70a8ee22fd1a9f89af3f9d469928b3c',
       '989678db8131e49fd29adc46c2e5e1a6e9ab6823203b37954a24b2b68a98083e',
       '682eb13643f153128c9c47afe6be08554cd042d0b38815b6745a2ec45b472a70',
       '2587a851f09c2bb4d22b7daf8ec30af164ad94a7f5d1620d60d271b85fff6f64',
       '27b0bf1efe11e967f3ddb1b1c2760eaff1659bc221ae60ba772c972de59bf4c1',
       'bcca3c58857d6029838ff9d34f3c576d76ac8dbccbe85c39961f0a9a8abe99e9',
       '7a4da56250e41de51576ffbfbc5db2aa6b38cc35867a20b4d28318a6e2c1c813',
       '775b40dfdcf5074b07b4c6639073b185e1133e0aaf468ede9ffd5fab9dd9943f',
       '6f6d4c27d6351b6dd1209847771f8aeda37ec301b11b442620e32bbda346bcb7',
       'f55a241cd8a27c7392b8ab9049495758f6895fefa5538e8aca2c27a204ecb0c3',
       '705d59380dfcee0454a0d3d9063900bf5e6706ab9f642b68a00f53099af0eafd',
       'c9c33a60aaec935cd9d3d1420c46c0e4bcec43463aac55dd454705b04f6a9f8c',
       'a14cbfea9a24e5720c98ad25ccdde36970b5ede89f9ac783a4bbebb8b61a1568',
       '58d466231ab231ae49c7b315b995b8e1b4916c0c3e79fe3c057b06ab8c3d6ec8',
       '497982be6706563d82b4f49446534877235c8f7f85099592e79d4da57a26d761',
       '005030a1caa49429f288b79917630d9a991690b95dde910c8f00a6b926e68c31',
       '22a6f0f1cec1769ea1d0898f041c36ebbfa5b1bf6c6b14d9d11b9119d02ae6ea',
       '316af6d1233ba85aa8e83c4defad35dd37cdc3d753700f5d16bd9da4b6af6ccd',
       'c2b4a8f51a1f1eddc7a459bbc3911da76708a878fcd0a16cb5bb6d2ab3dfb5d0',
       '204fa067ef1d8b8b034acfa1693f8c53e86c84305ee7503a199f40d3722ee57e',
       'ed91f04fbd495e2d5a00f327756aa4882f8b9d4f5d58b444150f5d89b2befebc',
       'eed1b477ab451eb20e847c90b3752b5a7d6c83d3b0c36285450f6731add2063e',
       'dff5c1910184b240d839f5a61143d74c7e537b1a7a6104c1878b69475e224b67',
       '5f78d8ea5d220fd4371a9d5b67f9704172b7e95aa85756242ea58a09ad6a2189',
       '84a578c10ef1a342ad03312393548815acba50632dd302b60f96e273abab64c8',
       '54b356aa258f805ac028589e97883ab66d011a00dd9e2a409eb5579f4e81abd4',
       '8eba4debb95ef877faed7041269483166a3145a7a590cba966da0a5aa394f2f3',
       '571fdff9a66b4534942db0655d35eef2dde903259323d6a36128a00a48b94efa',
       '68e9f31a207a753cc3460b17afd291ef4972d57cb149dbdac093fc4c986ad994',
       '802f787a8d4ebe3d4830943c5f4d50572512c76002ffdc4f9acaf238da85fb36',
       '645ee32ae1c84817505808b1a84e53565542cebd89d00d56207c3a8d111c60e5',
       '421548d50c73674295d827ff088f1d353441fdd64a0fa7ff74863cd8113bf991',
       'adbaa75ca91a86b069ec2810a47b226803cfcc90fb537c09a6945d9a13f19845',
       'a905175974a71113bb8794ec07b0e03add816a329c9e971c833710d4bdd52abf',
       'b85725fc8c972f70e3e8a290fc32b75df44bff1fbf31555ff512074003b6387b',
       '376ffc521ffbf5d3cb640e186b9db7a98795f7656a82b73207f6884ca1194589',
       'a6d0045c1c7ffb7db8fdcbe34d7ea0f46ce12606114b63b3702d312be308f9de']

In [6]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870,4,1.00000,1.000000,31.000000,77.500000,225.750000,4.0,1.0,1,1,1,0.250000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530,9326,236.40564,130.825005,78.920437,16523.837122,308234.733219,7070.0,26.0,47,74,1610,796.853957,528,582,519,540,512,417,421,626,360,346,223,308,207,272,844,136,298,175,259,359,259,323,397,415


In [7]:
def get_stats(a):
    x=0
    y=0
    for i in med:
        if i in m:
            x=x+a[a['Source_User']==i][0].values
            y=y+a[a['Source_User']==i]['Target'].values
    print("For high Severity:")
    print("identified:",x)
    print("Actual",y)
    print("For critical Severity")
    x=0
    y=0
    for i in cric:
        if i in m:
            x=x+a[a['Source_User']==i][0].values
            y=y+a[a['Source_User']==i]['Target'].values
    print("identified:",x)
    print("Actual",y)

## 80:20 Split

### random State 0

In [8]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870,4,1.00000,1.000000,31.000000,77.500000,225.750000,4.0,1.0,1,1,1,0.250000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530,9326,236.40564,130.825005,78.920437,16523.837122,308234.733219,7070.0,26.0,47,74,1610,796.853957,528,582,519,540,512,417,421,626,360,346,223,308,207,272,844,136,298,175,259,359,259,323,397,415


In [9]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870    0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530    1
c2cf660ad094544b4bb8f01efb417804cfa5d53624783c2f784faac2b244c029    1
3fcd1179a650c2cfc6b6d23aa0193f8247ea56691044389f53a1e771431e2862    0
55394724a0a52464de210c953c5e6bee779e0f0a163aa4787087024004caeb82    1
327686c9e4533b14641208d230f5d01b93dcdd21d71360fbcac0d78d487c58ab    0
bc72f41bcdc4b3e5ee020baae91eb97b6c4ca7a1209ed7a69f4f0915de990aa2    0
d49890688b35443d361df19e2bfeaeddeda4d5572c03cf5678f9b36e01cf5415    1
01c615bd6e5026eb8f8ce76567d9f00c3eec2b1752da1854ece27439da4c0fe9    0
469c1c634cf9e6412e08bda889e471ef6af563097f737223a09ad50e7364cfa9    0
Name: Target, dtype: int64

In [10]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)

The train/test split ratio is 80:20


### Bagging Classifier

In [11]:
bag=BaggingClassifier(n_estimators=500, max_samples=0.6,max_features=0.4,
                      bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)

bag.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(bag.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values


get_stats(a)

For high Severity:
identified: [12]
Actual [19]
For critical Severity
identified: [3]
Actual [3]


### GBC

In [12]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.3,
                               min_samples_leaf=20,subsample=0.5,random_state = 0, n_estimators = 150)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [10]
Actual [19]
For critical Severity
identified: [3]
Actual [3]


### Adaboost

In [13]:
ada=AdaBoostClassifier(n_estimators=50,learning_rate=1.5,algorithm='SAMME',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [10]
Actual [19]
For critical Severity
identified: [3]
Actual [3]


### Random State 10

In [14]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_10.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0d3fc59664e46c2fd425f7e198ccedc1faaffc4ae160fc7ec769d861466ae859,558,87.706093,25.094982,106.747312,3436.492832,103496.745520,539.0,20.0,42,47,230,7309.743728,5,4,0,0,0,0,4,31,14,3,3,4,16,18,20,74,81,71,116,42,32,10,4,6
384ed706777916e315308d41eb2efc086edc768b6392e14751b4d9bca760a59e,3,29.333333,22.000000,363.000000,5405.333333,3980.666667,3.0,1.0,2,1,2,0.000000,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
0d3fc59664e46c2fd425f7e198ccedc1faaffc4ae160fc7ec769d861466ae859    0
384ed706777916e315308d41eb2efc086edc768b6392e14751b4d9bca760a59e    0
25fbc1afa01f04686e25d7cffb238ca43ffeb865a4e6c1d07029f2fd8dcfcd32    1
b522e1c3ad002eddf9145e38f7f0a2be9c45895d718c7977e0a6a2a7becb3a10    1
8eba4debb95ef877faed7041269483166a3145a7a590cba966da0a5aa394f2f3    1
11e8b7b973c885d9d2560baeec4673bc6b13aab08d9e49bc9c519727595e4d3e    1
8389fdb8a5b9d6fff6c3eb065c2f0eeaeb6931647d7ff91de731a82a98c197c5    0
d0d0439bceb5f83d975bd909d12ad0f8a13766371c2a092a446febf0f7b08f5c    1
2587a851f09c2bb4d22b7daf8ec30af164ad94a7f5d1620d60d271b85fff6f64    1
841bc27abf403835526548adc7b881573c654cc78cb11b9e064c7b9682a89709    0
Name: Target, dtype: int64

In [16]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)

The train/test split ratio is 80:20


### Bagging Classifier

In [17]:
bag=BaggingClassifier(n_estimators=1500, max_samples=0.4,max_features=0.4,
                      bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)

bag.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(bag.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [4]
Actual [4]


### GBC

In [18]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.03, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=5,subsample=0.5,random_state = 0, n_estimators = 100)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [6]
Actual [9]
For critical Severity
identified: [4]
Actual [4]


### Adaboost

In [19]:
ada=AdaBoostClassifier(n_estimators=50,learning_rate=0.5,algorithm='SAMME.R',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [4]
Actual [4]


### Random State 20

In [20]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_20.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900,3728,132.185891,103.969421,135.190987,8652.968348,178992.604882,3096.0,264.0,44,114,784,2041.380633,72,81,115,106,131,116,100,355,279,152,196,153,135,443,243,195,185,132,152,107,90,107,38,45
24bcdf55cc1168a34c9d7534d7fb24500219538e85f2b8ddc3d1beabef89096b,81,31.061728,25.246914,109.395062,3871.506173,17973.666667,80.0,3.0,10,12,28,88488.987654,1,0,0,0,0,0,0,0,0,0,0,0,0,10,0,1,25,32,2,8,0,0,2,0


In [21]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900    1
24bcdf55cc1168a34c9d7534d7fb24500219538e85f2b8ddc3d1beabef89096b    0
c7ae13801caa67d372db28e31fe24146a34866c75fdaa7a8f3dd2e52c8feadbf    0
d7a31b1f8a1b6229ffc4f3d615d4847d8844645ed5bc9d4f016352ccde33eeef    0
2e8858be6011eed9b63eb701eda4575a43151dcedbd12b1e87e9707f1355e52d    1
aef14742e536c93ce48a00a70bc5c429ff5973e95f029488785eeda1778ea086    1
0930d98b6033a246eaecdf425d4fe74341a505bb2ffb83a09d4e489b29dea307    1
3f716f2fcc54a3eac07cc68af79773e6e32cc680c5f1b1e44ad527af39210d58    1
655f71c7531f608b3aa19880786cc023e26286b3bef2ad9917e379ddcead4e43    1
5957761ffa31d8e6cf65bb4540565c1ebd6e030e40b744a5c21d4d4b7c0d7d63    1
Name: Target, dtype: int64

In [22]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)

The train/test split ratio is 80:20


### Bagging Classifier

In [23]:
bag=BaggingClassifier(n_estimators=1500, max_samples=0.4,max_features=0.2,
                      bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)

bag.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(bag.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [2]
Actual [2]


### GBC

In [24]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.1, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=15,subsample=0.7,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [2]
Actual [2]


### Adaboost

In [25]:
ada=AdaBoostClassifier(n_estimators=50,learning_rate=0.5,algorithm='SAMME.R',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [2]
Actual [2]


## 70:30 Split

### Random state 0

In [26]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870,4,1.00000,1.000000,31.000000,77.500000,225.750000,4.0,1.0,1,1,1,0.250000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530,9326,236.40564,130.825005,78.920437,16523.837122,308234.733219,7070.0,26.0,47,74,1610,796.853957,528,582,519,540,512,417,421,626,360,346,223,308,207,272,844,136,298,175,259,359,259,323,397,415


In [27]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870    0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530    1
c2cf660ad094544b4bb8f01efb417804cfa5d53624783c2f784faac2b244c029    1
3fcd1179a650c2cfc6b6d23aa0193f8247ea56691044389f53a1e771431e2862    0
55394724a0a52464de210c953c5e6bee779e0f0a163aa4787087024004caeb82    1
327686c9e4533b14641208d230f5d01b93dcdd21d71360fbcac0d78d487c58ab    0
bc72f41bcdc4b3e5ee020baae91eb97b6c4ca7a1209ed7a69f4f0915de990aa2    0
d49890688b35443d361df19e2bfeaeddeda4d5572c03cf5678f9b36e01cf5415    1
01c615bd6e5026eb8f8ce76567d9f00c3eec2b1752da1854ece27439da4c0fe9    0
469c1c634cf9e6412e08bda889e471ef6af563097f737223a09ad50e7364cfa9    0
Name: Target, dtype: int64

In [28]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.3)

The train/test split ratio is 70:30


#### Kneighbor

In [31]:
knn=KNeighborsClassifier(algorithm='auto',leaf_size=2,metric='manhattan',n_neighbors=30,weights='uniform')

knn.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(knn.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values


get_stats(a)

For high Severity:
identified: [15]
Actual [24]
For critical Severity
identified: [5]
Actual [5]


### Adaboost

In [32]:
ada=AdaBoostClassifier(n_estimators=50,learning_rate=0.5,algorithm='SAMME.R',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [15]
Actual [24]
For critical Severity
identified: [5]
Actual [5]


### GBC

In [33]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.2, criterion='mse', min_samples_split=0.3,
                               min_samples_leaf=10,subsample=0.7,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [14]
Actual [24]
For critical Severity
identified: [5]
Actual [5]


### Bagging Classifier

In [34]:
bag=BaggingClassifier(n_estimators=1000, max_samples=0.3,max_features=0.6,
                      bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)

bag.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(bag.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values


get_stats(a)

For high Severity:
identified: [16]
Actual [24]
For critical Severity
identified: [5]
Actual [5]


### Random State 10

In [35]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_10.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0d3fc59664e46c2fd425f7e198ccedc1faaffc4ae160fc7ec769d861466ae859,558,87.706093,25.094982,106.747312,3436.492832,103496.745520,539.0,20.0,42,47,230,7309.743728,5,4,0,0,0,0,4,31,14,3,3,4,16,18,20,74,81,71,116,42,32,10,4,6
384ed706777916e315308d41eb2efc086edc768b6392e14751b4d9bca760a59e,3,29.333333,22.000000,363.000000,5405.333333,3980.666667,3.0,1.0,2,1,2,0.000000,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Kneighbor

In [36]:
knn=KNeighborsClassifier(algorithm='auto',leaf_size=2,metric='manhattan',n_neighbors=50,weights='distance')

knn.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(knn.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values


get_stats(a)

For high Severity:
identified: [15]
Actual [24]
For critical Severity
identified: [5]
Actual [5]


### Adaboost

In [37]:
ada=AdaBoostClassifier(n_estimators=150,learning_rate=1,algorithm='SAMME',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [16]
Actual [24]
For critical Severity
identified: [5]
Actual [5]


### GBC

In [38]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.2,
                               min_samples_leaf=5,subsample=0.7,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [15]
Actual [24]
For critical Severity
identified: [5]
Actual [5]


### Bagging Classifier

In [39]:
bag=BaggingClassifier(n_estimators=1500, max_samples=0.3,max_features=0.6,
                      bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)

bag.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(bag.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values


get_stats(a)

For high Severity:
identified: [16]
Actual [24]
For critical Severity
identified: [5]
Actual [5]


### Random State 20

In [40]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_20.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900,3728,132.185891,103.969421,135.190987,8652.968348,178992.604882,3096.0,264.0,44,114,784,2041.380633,72,81,115,106,131,116,100,355,279,152,196,153,135,443,243,195,185,132,152,107,90,107,38,45
24bcdf55cc1168a34c9d7534d7fb24500219538e85f2b8ddc3d1beabef89096b,81,31.061728,25.246914,109.395062,3871.506173,17973.666667,80.0,3.0,10,12,28,88488.987654,1,0,0,0,0,0,0,0,0,0,0,0,0,10,0,1,25,32,2,8,0,0,2,0


In [41]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900    1
24bcdf55cc1168a34c9d7534d7fb24500219538e85f2b8ddc3d1beabef89096b    0
c7ae13801caa67d372db28e31fe24146a34866c75fdaa7a8f3dd2e52c8feadbf    0
d7a31b1f8a1b6229ffc4f3d615d4847d8844645ed5bc9d4f016352ccde33eeef    0
2e8858be6011eed9b63eb701eda4575a43151dcedbd12b1e87e9707f1355e52d    1
aef14742e536c93ce48a00a70bc5c429ff5973e95f029488785eeda1778ea086    1
0930d98b6033a246eaecdf425d4fe74341a505bb2ffb83a09d4e489b29dea307    1
3f716f2fcc54a3eac07cc68af79773e6e32cc680c5f1b1e44ad527af39210d58    1
655f71c7531f608b3aa19880786cc023e26286b3bef2ad9917e379ddcead4e43    1
5957761ffa31d8e6cf65bb4540565c1ebd6e030e40b744a5c21d4d4b7c0d7d63    1
Name: Target, dtype: int64

In [42]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.3)

The train/test split ratio is 70:30


#### Kneighbor

In [43]:
knn=KNeighborsClassifier(algorithm='auto',leaf_size=2,metric='manhattan',n_neighbors=50,weights='distance')

knn.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(knn.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values


get_stats(a)

For high Severity:
identified: [13]
Actual [16]
For critical Severity
identified: [2]
Actual [3]


### Adaboost

In [44]:
ada=AdaBoostClassifier(n_estimators=50,learning_rate=0.5,algorithm='SAMME.R',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [13]
Actual [16]
For critical Severity
identified: [2]
Actual [3]


### GBC

In [45]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.1, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=15,subsample=0.7,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [13]
Actual [16]
For critical Severity
identified: [2]
Actual [3]


### Bagging Classifier

In [46]:
bag=BaggingClassifier(n_estimators=500, max_samples=0.4,max_features=0.2,
                      bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)

bag.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(bag.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values


get_stats(a)

For high Severity:
identified: [13]
Actual [16]
For critical Severity
identified: [2]
Actual [3]


### 80:20 with PCA

#### Random State 0

In [47]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870,4,1.00000,1.000000,31.000000,77.500000,225.750000,4.0,1.0,1,1,1,0.250000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530,9326,236.40564,130.825005,78.920437,16523.837122,308234.733219,7070.0,26.0,47,74,1610,796.853957,528,582,519,540,512,417,421,626,360,346,223,308,207,272,844,136,298,175,259,359,259,323,397,415


In [48]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870    0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530    1
c2cf660ad094544b4bb8f01efb417804cfa5d53624783c2f784faac2b244c029    1
3fcd1179a650c2cfc6b6d23aa0193f8247ea56691044389f53a1e771431e2862    0
55394724a0a52464de210c953c5e6bee779e0f0a163aa4787087024004caeb82    1
327686c9e4533b14641208d230f5d01b93dcdd21d71360fbcac0d78d487c58ab    0
bc72f41bcdc4b3e5ee020baae91eb97b6c4ca7a1209ed7a69f4f0915de990aa2    0
d49890688b35443d361df19e2bfeaeddeda4d5572c03cf5678f9b36e01cf5415    1
01c615bd6e5026eb8f8ce76567d9f00c3eec2b1752da1854ece27439da4c0fe9    0
469c1c634cf9e6412e08bda889e471ef6af563097f737223a09ad50e7364cfa9    0
Name: Target, dtype: int64

In [49]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.2)

Variance explained using 13 components is: 0.918903941373
The train/test split ratio is 80:20


### GBC

In [50]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.03, criterion='mse', min_samples_split=0.2,
                               min_samples_leaf=15,subsample=1,random_state = 0, n_estimators = 150)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [12]
Actual [19]
For critical Severity
identified: [3]
Actual [3]


### NN2

In [51]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [11]
Actual [19]
For critical Severity
identified: [3]
Actual [3]


### Random State 10

In [52]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_10.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0d3fc59664e46c2fd425f7e198ccedc1faaffc4ae160fc7ec769d861466ae859,558,87.706093,25.094982,106.747312,3436.492832,103496.745520,539.0,20.0,42,47,230,7309.743728,5,4,0,0,0,0,4,31,14,3,3,4,16,18,20,74,81,71,116,42,32,10,4,6
384ed706777916e315308d41eb2efc086edc768b6392e14751b4d9bca760a59e,3,29.333333,22.000000,363.000000,5405.333333,3980.666667,3.0,1.0,2,1,2,0.000000,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
0d3fc59664e46c2fd425f7e198ccedc1faaffc4ae160fc7ec769d861466ae859    0
384ed706777916e315308d41eb2efc086edc768b6392e14751b4d9bca760a59e    0
25fbc1afa01f04686e25d7cffb238ca43ffeb865a4e6c1d07029f2fd8dcfcd32    1
b522e1c3ad002eddf9145e38f7f0a2be9c45895d718c7977e0a6a2a7becb3a10    1
8eba4debb95ef877faed7041269483166a3145a7a590cba966da0a5aa394f2f3    1
11e8b7b973c885d9d2560baeec4673bc6b13aab08d9e49bc9c519727595e4d3e    1
8389fdb8a5b9d6fff6c3eb065c2f0eeaeb6931647d7ff91de731a82a98c197c5    0
d0d0439bceb5f83d975bd909d12ad0f8a13766371c2a092a446febf0f7b08f5c    1
2587a851f09c2bb4d22b7daf8ec30af164ad94a7f5d1620d60d271b85fff6f64    1
841bc27abf403835526548adc7b881573c654cc78cb11b9e064c7b9682a89709    0
Name: Target, dtype: int64

In [54]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.2)

Variance explained using 13 components is: 0.920248168257
The train/test split ratio is 80:20


### GBC

In [55]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.03, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=20,subsample=1,random_state = 0, n_estimators = 100)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [4]
Actual [4]


### NN2

In [56]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [4]
Actual [4]


### Random State 20

In [57]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_20.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900,3728,132.185891,103.969421,135.190987,8652.968348,178992.604882,3096.0,264.0,44,114,784,2041.380633,72,81,115,106,131,116,100,355,279,152,196,153,135,443,243,195,185,132,152,107,90,107,38,45
24bcdf55cc1168a34c9d7534d7fb24500219538e85f2b8ddc3d1beabef89096b,81,31.061728,25.246914,109.395062,3871.506173,17973.666667,80.0,3.0,10,12,28,88488.987654,1,0,0,0,0,0,0,0,0,0,0,0,0,10,0,1,25,32,2,8,0,0,2,0


In [58]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900    1
24bcdf55cc1168a34c9d7534d7fb24500219538e85f2b8ddc3d1beabef89096b    0
c7ae13801caa67d372db28e31fe24146a34866c75fdaa7a8f3dd2e52c8feadbf    0
d7a31b1f8a1b6229ffc4f3d615d4847d8844645ed5bc9d4f016352ccde33eeef    0
2e8858be6011eed9b63eb701eda4575a43151dcedbd12b1e87e9707f1355e52d    1
aef14742e536c93ce48a00a70bc5c429ff5973e95f029488785eeda1778ea086    1
0930d98b6033a246eaecdf425d4fe74341a505bb2ffb83a09d4e489b29dea307    1
3f716f2fcc54a3eac07cc68af79773e6e32cc680c5f1b1e44ad527af39210d58    1
655f71c7531f608b3aa19880786cc023e26286b3bef2ad9917e379ddcead4e43    1
5957761ffa31d8e6cf65bb4540565c1ebd6e030e40b744a5c21d4d4b7c0d7d63    1
Name: Target, dtype: int64

In [59]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.2)

Variance explained using 13 components is: 0.92323693469
The train/test split ratio is 80:20


### GBC

In [60]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.03, criterion='mse', min_samples_split=0.2,
                               min_samples_leaf=10,subsample=1,random_state = 0, n_estimators = 100)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [2]
Actual [2]


### NN2

In [61]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [7]
Actual [9]
For critical Severity
identified: [2]
Actual [2]


### 70:30 Split with PCA

#### Random State 0

In [62]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870,4,1.00000,1.000000,31.000000,77.500000,225.750000,4.0,1.0,1,1,1,0.250000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530,9326,236.40564,130.825005,78.920437,16523.837122,308234.733219,7070.0,26.0,47,74,1610,796.853957,528,582,519,540,512,417,421,626,360,346,223,308,207,272,844,136,298,175,259,359,259,323,397,415


In [63]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
21f9a46d094a0039eac8b09f38e2a760d7977c809ec91a1193b2d05e46c2e870    0
705f5e8c54520cba59aa4307a315b09f35dd4138efb687c1f6ca9d117f8b3530    1
c2cf660ad094544b4bb8f01efb417804cfa5d53624783c2f784faac2b244c029    1
3fcd1179a650c2cfc6b6d23aa0193f8247ea56691044389f53a1e771431e2862    0
55394724a0a52464de210c953c5e6bee779e0f0a163aa4787087024004caeb82    1
327686c9e4533b14641208d230f5d01b93dcdd21d71360fbcac0d78d487c58ab    0
bc72f41bcdc4b3e5ee020baae91eb97b6c4ca7a1209ed7a69f4f0915de990aa2    0
d49890688b35443d361df19e2bfeaeddeda4d5572c03cf5678f9b36e01cf5415    1
01c615bd6e5026eb8f8ce76567d9f00c3eec2b1752da1854ece27439da4c0fe9    0
469c1c634cf9e6412e08bda889e471ef6af563097f737223a09ad50e7364cfa9    0
Name: Target, dtype: int64

In [64]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.3)

Variance explained using 13 components is: 0.918903941373
The train/test split ratio is 70:30


### GBC

In [66]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.5,
                               min_samples_leaf=10,subsample=0.1,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [0]
Actual [24]
For critical Severity
identified: [0]
Actual [5]


### Extra Tree

In [67]:
ext_tree=ExtraTreeClassifier(criterion='entropy',max_depth=10,max_features='auto',min_impurity_decrease=1e-3,
                             min_samples_leaf=2,min_samples_split=2,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [14]
Actual [24]
For critical Severity
identified: [4]
Actual [5]


### Random State 10

In [68]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_10.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0d3fc59664e46c2fd425f7e198ccedc1faaffc4ae160fc7ec769d861466ae859,558,87.706093,25.094982,106.747312,3436.492832,103496.745520,539.0,20.0,42,47,230,7309.743728,5,4,0,0,0,0,4,31,14,3,3,4,16,18,20,74,81,71,116,42,32,10,4,6
384ed706777916e315308d41eb2efc086edc768b6392e14751b4d9bca760a59e,3,29.333333,22.000000,363.000000,5405.333333,3980.666667,3.0,1.0,2,1,2,0.000000,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
0d3fc59664e46c2fd425f7e198ccedc1faaffc4ae160fc7ec769d861466ae859    0
384ed706777916e315308d41eb2efc086edc768b6392e14751b4d9bca760a59e    0
25fbc1afa01f04686e25d7cffb238ca43ffeb865a4e6c1d07029f2fd8dcfcd32    1
b522e1c3ad002eddf9145e38f7f0a2be9c45895d718c7977e0a6a2a7becb3a10    1
8eba4debb95ef877faed7041269483166a3145a7a590cba966da0a5aa394f2f3    1
11e8b7b973c885d9d2560baeec4673bc6b13aab08d9e49bc9c519727595e4d3e    1
8389fdb8a5b9d6fff6c3eb065c2f0eeaeb6931647d7ff91de731a82a98c197c5    0
d0d0439bceb5f83d975bd909d12ad0f8a13766371c2a092a446febf0f7b08f5c    1
2587a851f09c2bb4d22b7daf8ec30af164ad94a7f5d1620d60d271b85fff6f64    1
841bc27abf403835526548adc7b881573c654cc78cb11b9e064c7b9682a89709    0
Name: Target, dtype: int64

In [70]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.3)

Variance explained using 13 components is: 0.920248168257
The train/test split ratio is 70:30


### GBC

In [71]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.05, criterion='mse', min_samples_split=0.5,
                               min_samples_leaf=10,subsample=0.1,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [0]
Actual [16]
For critical Severity
identified: [0]
Actual [6]


### Extra Tree

In [72]:
ext_tree=ExtraTreeClassifier(criterion='entropy',max_depth=10,max_features='auto',min_impurity_decrease=1e-3,
                             min_samples_leaf=2,min_samples_split=2,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [13]
Actual [16]
For critical Severity
identified: [4]
Actual [6]


### Random State 20

In [73]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_50_50_2_27_random_state_20.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


infected      2481
uninfected    2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900,3728,132.185891,103.969421,135.190987,8652.968348,178992.604882,3096.0,264.0,44,114,784,2041.380633,72,81,115,106,131,116,100,355,279,152,196,153,135,443,243,195,185,132,152,107,90,107,38,45
24bcdf55cc1168a34c9d7534d7fb24500219538e85f2b8ddc3d1beabef89096b,81,31.061728,25.246914,109.395062,3871.506173,17973.666667,80.0,3.0,10,12,28,88488.987654,1,0,0,0,0,0,0,0,0,0,0,0,0,10,0,1,25,32,2,8,0,0,2,0


In [74]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900    1
24bcdf55cc1168a34c9d7534d7fb24500219538e85f2b8ddc3d1beabef89096b    0
c7ae13801caa67d372db28e31fe24146a34866c75fdaa7a8f3dd2e52c8feadbf    0
d7a31b1f8a1b6229ffc4f3d615d4847d8844645ed5bc9d4f016352ccde33eeef    0
2e8858be6011eed9b63eb701eda4575a43151dcedbd12b1e87e9707f1355e52d    1
aef14742e536c93ce48a00a70bc5c429ff5973e95f029488785eeda1778ea086    1
0930d98b6033a246eaecdf425d4fe74341a505bb2ffb83a09d4e489b29dea307    1
3f716f2fcc54a3eac07cc68af79773e6e32cc680c5f1b1e44ad527af39210d58    1
655f71c7531f608b3aa19880786cc023e26286b3bef2ad9917e379ddcead4e43    1
5957761ffa31d8e6cf65bb4540565c1ebd6e030e40b744a5c21d4d4b7c0d7d63    1
Name: Target, dtype: int64

In [75]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.3)

Variance explained using 13 components is: 0.92323693469
The train/test split ratio is 70:30


### GBC

In [76]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.03, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=10,subsample=0.5,random_state = 0, n_estimators = 100)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [13]
Actual [16]
For critical Severity
identified: [2]
Actual [3]


### Extra Tree

In [77]:
ext_tree=ExtraTreeClassifier(criterion='gini',max_depth=20,max_features='auto',min_impurity_decrease=1e-3,
                             min_samples_leaf=3,min_samples_split=2,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [13]
Actual [16]
For critical Severity
identified: [1]
Actual [3]


In [78]:
### Original Creator : Darshan Bhansali
### HTML code to hide the input cells 
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle code, click <a href="javascript:code_toggle()">here</a>.''')